In [1]:
from dotenv import load_dotenv
import warnings

from environment.game import Game
from environment.game_state import GameState
from environment.generators.random_generator import RandomCrimeSceneMapGenerator
from llm.llama import Llama
from llm.story_generator import StoryGenerator

load_dotenv(dotenv_path="./llm-dungeon-adventures/.env")
warnings.filterwarnings("ignore", category=FutureWarning)

/home/jovyan/my-conda-envs/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1. Create a room layout and a theme

In [2]:
number_of_rooms = 3
theme = "Smallville, Clark Kent, 2010"

crime_scene_map = RandomCrimeSceneMapGenerator().generate(number_of_rooms, 42)

2. Init Llama2

In [3]:
from langchain.llms import HuggingFacePipeline

llm = Llama()
llm_pipeline = HuggingFacePipeline(pipeline=llm.pipeline)

Loading checkpoint shards: 100%|██████████| 3/3 [00:32<00:00, 10.78s/it]


Model loaded on cuda:0


In [9]:
# import os, transformers

# hf_auth = os.environ["HF_AUTH"]
# tokenizer = transformers.AutoTokenizer.from_pretrained(
#     "meta-llama/Llama-2-13b-chat-hf", use_auth_token=hf_auth
# )

# len(
#     tokenizer.tokenize(
#         """<s>[INST] <<SYS>>
            
#             You are a crime storyteller. Always output answer as an array of JSON objects of this scheme: {"type": "array", "items": {"type": "object", "properties": {"name": {"type": "string"}, "age": {"type": "number"}, "occupation": {"type": "string"}, "alibi": {"type": "string"}, "motive": {"type": "string"}}, "required": ["name", "age", "occupation", "alibi", "motive"]}}.
#             Avoid outputting anything else than the array of JSON objects.
            
#             <<SYS>>

#             Given a theme: Library of Alexandria, 340 BC, crazy librarian, victim information: {"name": "Archibald Ptolemy", "age": 35, "occupation": "Head Librarian", "murder_weapon": "Ancient scroll with poisoned ink", "death_description": "Found dead in his office surrounded by stacks of books, face contorted in a mixture of fear and surprise, as if he had been reading a particularly gruesome text when struck down."} and killer information: {"name": "Gaius", "age": 40, "occupation": "Crazy librarian", "alibi": "Gaius has no alibi. He claims he was in his secret chamber, delving into forbidden texts. No one can vouch for his whereabouts.", "motive": "Gaius had become increasingly obsessed with ancient and forbidden knowledge. He believed that by eliminating anyone who questioned him, he could protect the library's secrets."}, describe 2 suspects that are not the killer. Avoid nicknames.
#             suspects:
#             [/INST]
#             [{"name": "Lucius", "age": 35, "occupation": "Librarian's Assistant", "alibi": "Lucius claims he was organizing scrolls in the library's main hall at the time of the murder. Several witnesses saw him there throughout the evening.", "motive": "Lucius had a longstanding feud with Drusilla, who constantly criticized his work and suggested he was not fit for his role. He might have wanted to silence her."}, {"name": "Cassandra", "age": 28, "occupation": "Junior Librarian", "motive": "Cassandra felt threatened by Archibald's strict rules and constant criticism of her work. She may have sought revenge through this brutal act.", "alibi": "Cassandra asserts she was searching for lost texts in the stacks during the homicide. Multiple patrons corroborate her presence near the scene around the estimated time of death."}]</s><s>
            
#             [INST]
#             Given a theme: ['Prohibition Era', 'Speakeasy', 'Bootlegger'], victim information: null and killer information: null, describe 2 suspect that are not the killer. Avoid nicknames.
#             suspects:
#             [/INST]"""
#     )
# )

702

In [4]:
import json
import re
from time import time

from llm.chains.killer_chain import KillerChain
from llm.chains.suspect_chain import SuspectChain
from llm.chains.victim_chain import VictimChain

themes = [
    ['Prohibition Era', 'Speakeasy', 'Bootlegger'],
    # ['Victorian London', 'Jack the Ripper', 'Scotland Yard'],
    # ['1920s Chicago', 'Al Capone', 'Organized Crime'],
    # ['Ancient Rome', 'Emperor Nero', 'Assassination Plot'],
    # ['Futuristic Megacity', 'Cybernetic Hacker', 'Corporate Espionage'],
    # ['Medieval Castle', 'Royal Poisoning', 'Intrigue'],
    # ['Wild West', 'Outlaw Gang', 'Bank Heist'],
    # ['Small Coastal Town', 'Missing Fisherman', 'Mysterious Disappearances'],
    # ['Art World Heist', 'Stolen Masterpiece', 'Undercover Detective'],
    # ['1940s Hollywood', 'Film Noir Detective', 'Murder Mystery'],
]

killer_chain, victim_chain, suspect_chain = KillerChain(llm_pipeline), VictimChain(llm_pipeline), SuspectChain(llm_pipeline)

results = []
for theme in themes:
    start_time = time()
    victim = victim_chain.create(theme)
    #killer = killer_chain.create(theme, victim)
    #suspects = suspect_chain.create(theme, victim, killer)
    end_time = time()
    
    results.append({
        "elapsed_time": end_time - start_time,
        "theme": theme,
        "victim": victim,
        #"killer": killer,
        #"suspects": suspects
    })

In [6]:
results

[{'elapsed_time': 9.588946104049683,
  'theme': ['Prohibition Era', 'Speakeasy', 'Bootlegger'],
  'victim': None}]

3. Create a story

In [ ]:
story = StoryGenerator(crime_scene_map.rooms, llm).create_new_story(theme=theme, dummy=False)

story

4. Create an initial game state

In [ ]:
initial_game_state = GameState(crime_scene_map, story)

5. Create a game

In [ ]:
game = Game(initial_game_state)